# Clasificación de estado de presión de llantas vehículos a partir de imágenes

###Juan Nicolás Soto Rios
###Katherine Xiomar González Santacruz
###Frank M. Córdoba O.


# 1. Cargando Archivos

In [ ]:
import os
os.getcwd()
import cv2
from google.colab.patches import cv2_imshow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
flat_class = []
for image in os.listdir('flat.class'):
  flat_class.append(cv2.imread(os.path.join('flat.class', image)))

In [ ]:
full_class = []
for image in os.listdir('full.class'):
  full_class.append(cv2.imread(os.path.join('full.class', image)))

In [ ]:
cv2_imshow(flat_class[0]), cv2_imshow(full_class[0]) 

In [ ]:
print(flat_class[0].shape)
print(len(flat_class))

print(full_class[0].shape)
print(len(full_class))

In [ ]:
flat_gray = []
full_gray = []
full_data = []
for img in flat_class:
  flat_gray.append(np.array(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)).flatten())
  full_data.append(np.array(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)).flatten())
for img in full_class:
  full_gray.append(np.array(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)).flatten())
  full_data.append(np.array(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)).flatten())

flat_gray = np.array(flat_gray)
full_gray = np.array(full_gray)
full_data = np.array(full_data)

In [ ]:
print(flat_gray.shape)
print(len(flat_gray))

print(full_gray.shape)
print(len(full_gray))

print(full_data.shape)
print(len(full_data))

In [ ]:
etiquetas = np.zeros((300,))
etiquetas = np.append(etiquetas, np.ones((300,)))
etiquetas.shape

## 2. Analizar dimensionalidad de los datos

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=200)

pca.fit(full_data)

In [ ]:
print(pca.explained_variance_ratio_[:5])
print(pca.explained_variance_ratio_.sum())

In [ ]:
# Plot
c = []
v_ac = []
v_i = 0
for v in pca.explained_variance_ratio_:
  v_i += v
  v_ac.append(v_i) 
plt.plot(range(len(pca.explained_variance_ratio_)), v_ac)
plt.ylabel('Varianza Explicada')
plt.xlabel('Componentes Principales')
plt.title('Varianza Explicada Acumulada')
plt.show()

In [ ]:
n_c = 0
i = 0
counting = True
while counting:
  n_c += pca.explained_variance_ratio_[i]
  i += 1
  if n_c >= 0.8:
    print(f'Numero de variables: {i}')
    counting = False
  

In [ ]:
pca = PCA(3) 
projected = pca.fit_transform(full_data)

plt.scatter(projected[:, 0], projected[:, 1],
            c=etiquetas, edgecolor='none', alpha=0.5, cmap=plt.cm.get_cmap('viridis', 2))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();

In [ ]:
import plotly.express as px
fig = px.scatter_3d( x= projected[:, 0], y= projected[:, 1], z= projected[:, 2],
              color= etiquetas, )
fig.show()

In [ ]:
pca = PCA(70)

X = pca.fit_transform(full_data)
y = etiquetas

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test_p, y_train, y_test_p = train_test_split(X, y, test_size=0.3, random_state=42)


print(X_train.shape)
print(X_test_p.shape)
print(y_train.shape)
print(y_test_p.shape)

## 3. Red Neuronal

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

X_val, X_test, y_val, y_test = train_test_split(X_test_p, y_test_p, test_size= 0.3, random_state= 42)

validation = True

n_features = X_train.shape[1]
# Definicion del modelo
nn_model = Sequential()
nn_model.add(Dense(128, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
nn_model.add(Dense(64, activation='relu', kernel_initializer='he_normal'))
nn_model.add(Dense(32, activation='relu', kernel_initializer='he_normal'))
nn_model.add(Dense(1, activation='sigmoid'))

# Callback
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# Compilacion del modelo
nn_model.compile(optimizer='adam', loss='binaryy _crossentropy', metrics=['accuracy'])

# fit
if validation: 
  nn_model.fit(X_train, y_train, epochs=20, batch_size=10,  callbacks=[callback],  validation_data = (X_val, y_val))
else:
   nn_model.fit(X_train, y_train, epochs=20, batch_size=10,  callbacks=[callback],  validation_data = None)
# evaluacion
if validation:
  # train accuracy
  loss, acc = nn_model.evaluate(X_train, y_train, verbose=0)
  print('Train Accuracy: %.3f' % acc)
  # test acccuracy
  loss, acc = nn_model.evaluate(X_test, y_test, verbose=0)
  print('Test Accuracy: %.3f' % acc)
else:
  # train accuracy
  loss, acc = nn_model.evaluate(X_train, y_train, verbose=0)
  print('Train Accuracy: %.3f' % acc)
  # test acccuracy
  loss, acc = nn_model.evaluate(X_test_p, y_test_p, verbose=0)
  print('Test Accuracy: %.3f' % acc)

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(nn_model,  show_shapes=True, show_layer_names=True)

In [ ]:
y_pred_nn = nn_model.predict(X_test_p)
y_pred_nn = np.array(y_pred_nn).flatten()
y_pred_nn = (y_pred_nn > 0.9).astype(int)
print(classification_report(y_test_p, y_pred_nn))

In [ ]:
cm = confusion_matrix(y_test_p, y_pred_nn)
print(cm)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels= None)


# NOTE: Fill all variables here with default values of the plot_confusion_matrix
disp = disp.plot(include_values= True, cmap='viridis')

plt.show()